# 🎛️ Day 3: Hyperparameter Tuning

**🎯 Goal:** Find the best settings for your AI models to maximize performance

**⏱️ Time:** 45-60 minutes

**🌟 Why This Matters for AI:**
- Default parameters are rarely optimal for your specific problem
- Proper tuning can improve accuracy by 5-20%!
- Essential for Kaggle competitions, research papers, production systems
- Used to optimize GPT, BERT, transformers, and all modern AI
- Difference between good model and winning model

---

## 🤔 What Are Hyperparameters?

**Parameters:** Learned from data during training
- Example: Weights in neural networks
- Model learns these automatically

**Hyperparameters:** Set BEFORE training
- Example: Number of trees in Random Forest
- YOU must choose these
- Huge impact on performance!

**Examples:**
- Random Forest: `n_estimators`, `max_depth`, `min_samples_split`
- Neural Networks: `learning_rate`, `batch_size`, `num_layers`
- GPT/Transformers: `num_heads`, `hidden_size`, `dropout_rate`

**The Challenge:** How do you find the BEST combination? 🎯

In [ ]:
# Import our tools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score
)
from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import time
from scipy.stats import randint, uniform

# Set style and random seed
sns.set_style('whitegrid')
np.random.seed(42)

print("✅ Libraries imported successfully!")

## ❌ The Naive Approach: Manual Trial & Error

Let's see why manual tuning is painful:

In [ ]:
# Load dataset
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("🔬 MANUAL HYPERPARAMETER TUNING (The Hard Way)")
print("=" * 60)

# Try different combinations manually
configs = [
    {'n_estimators': 10, 'max_depth': 5},
    {'n_estimators': 50, 'max_depth': 10},
    {'n_estimators': 100, 'max_depth': 15},
    {'n_estimators': 200, 'max_depth': 20},
]

results = []
for config in configs:
    model = RandomForestClassifier(**config, random_state=42)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    results.append(score)
    print(f"n_estimators={config['n_estimators']:3d}, max_depth={config['max_depth']:2d} → Accuracy: {score:.4f}")

print("\n⚠️  Problems with Manual Tuning:")
print("   1. Time-consuming (tried only 4 combinations!)")
print("   2. May miss the best combination")
print("   3. No systematic search")
print("   4. Hard to explore many parameters")
print("\n💡 Solution: Automated Hyperparameter Tuning! 🚀")

## 🔍 Grid Search: Exhaustive Search

**How it works:**
1. Define a grid of hyperparameter values
2. Try EVERY combination
3. Use cross-validation for each
4. Return the best combination

**Example:**
```python
param_grid = {
    'n_estimators': [10, 50, 100],      # 3 values
    'max_depth': [5, 10, 15, 20]        # 4 values
}
# Total combinations: 3 × 4 = 12
# With 5-fold CV: 12 × 5 = 60 model trainings!
```

**Pros:**
- ✅ Guarantees finding the best combination in the grid
- ✅ Simple and straightforward
- ✅ Good for few parameters

**Cons:**
- ❌ Exponentially slow (curse of dimensionality)
- ❌ Wastes time on bad regions

In [ ]:
# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print("🔍 GRID SEARCH")
print("=" * 60)
print(f"Parameter grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

total_combinations = np.prod([len(v) for v in param_grid.values()])
print(f"\nTotal combinations: {total_combinations}")
print(f"With 5-fold CV: {total_combinations * 5} model trainings!\n")

# Perform Grid Search
print("⏳ Running Grid Search (this may take a minute)...\n")

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,  # Use all CPU cores
    verbose=1
)

start_time = time.time()
grid_search.fit(X_train, y_train)
grid_time = time.time() - start_time

print(f"\n✅ Grid Search completed in {grid_time:.2f} seconds")
print("\n" + "=" * 60)
print("🏆 BEST HYPERPARAMETERS FOUND:")
print("=" * 60)
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\n📊 Best CV Score: {grid_search.best_score_:.4f}")
print(f"📊 Test Score: {grid_search.score(X_test, y_test):.4f}")

## 📊 Visualizing Grid Search Results

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Display top 10 configurations
print("🎯 TOP 10 CONFIGURATIONS:\n")
top_10 = results_df.nsmallest(10, 'rank_test_score')[[
    'param_n_estimators', 'param_max_depth', 'param_min_samples_split',
    'param_min_samples_leaf', 'mean_test_score', 'std_test_score'
]]

print(top_10.to_string(index=False))

# Visualize: n_estimators vs max_depth
pivot_table = results_df.pivot_table(
    values='mean_test_score',
    index='param_max_depth',
    columns='param_n_estimators',
    aggfunc='mean'
)

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='YlGnBu', cbar_kws={'label': 'Accuracy'})
plt.title('Grid Search: n_estimators vs max_depth (averaged over other params)', 
         fontsize=14, fontweight='bold')
plt.xlabel('n_estimators', fontsize=12)
plt.ylabel('max_depth', fontsize=12)
plt.tight_layout()
plt.show()

print("\n💡 Insights from Heatmap:")
print("   - Darker blue = better performance")
print("   - Can see which parameter ranges work best")
print("   - Helps understand parameter interactions")

## 🎲 Random Search: Smarter Exploration

**Problem with Grid Search:**
- If you have 5 parameters with 10 values each: 10^5 = 100,000 combinations!
- Completely impractical

**Random Search Solution:**
- Instead of trying ALL combinations, sample randomly
- You decide how many combinations to try (e.g., 50)
- Often finds good solutions faster!

**Research Finding (Bergstra & Bengio, 2012):**
- Random Search often outperforms Grid Search
- Better explores the hyperparameter space
- More efficient for high-dimensional spaces

**Real AI Use:**
- Standard for deep learning (too many hyperparameters)
- BERT, GPT training uses variants of random search
- Recommended by Google, OpenAI

In [ ]:
# Define parameter distributions (not fixed values!)
param_distributions = {
    'n_estimators': randint(50, 300),           # Random integers between 50 and 300
    'max_depth': randint(5, 30),                # Random integers between 5 and 30
    'min_samples_split': randint(2, 20),        # Random integers between 2 and 20
    'min_samples_leaf': randint(1, 10),         # Random integers between 1 and 10
    'max_features': uniform(0.1, 0.9)           # Random float between 0.1 and 1.0
}

print("🎲 RANDOM SEARCH")
print("=" * 60)
print(f"Parameter distributions:")
for param, dist in param_distributions.items():
    print(f"  {param}: {dist}")

n_iterations = 50
print(f"\nTrying {n_iterations} random combinations")
print(f"With 5-fold CV: {n_iterations * 5} model trainings\n")

# Perform Random Search
print("⏳ Running Random Search...\n")

random_search = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions,
    n_iter=n_iterations,  # Number of random combinations to try
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

start_time = time.time()
random_search.fit(X_train, y_train)
random_time = time.time() - start_time

print(f"\n✅ Random Search completed in {random_time:.2f} seconds")
print(f"⚡ Grid Search took {grid_time:.2f} seconds")
print(f"🚀 Random Search was {grid_time/random_time:.2f}x faster!\n")

print("=" * 60)
print("🏆 BEST HYPERPARAMETERS FOUND:")
print("=" * 60)
for param, value in random_search.best_params_.items():
    if isinstance(value, float):
        print(f"  {param}: {value:.4f}")
    else:
        print(f"  {param}: {value}")

print(f"\n📊 Best CV Score: {random_search.best_score_:.4f}")
print(f"📊 Test Score: {random_search.score(X_test, y_test):.4f}")

# Compare with Grid Search
print("\n⚖️  GRID SEARCH vs RANDOM SEARCH:")
print("=" * 60)
print(f"{'Method':<20} {'Best CV Score':<15} {'Time (s)':<10} {'Evaluations'}")
print("-" * 60)
print(f"{'Grid Search':<20} {grid_search.best_score_:<15.4f} {grid_time:<10.2f} {len(grid_search.cv_results_['params'])}")
print(f"{'Random Search':<20} {random_search.best_score_:<15.4f} {random_time:<10.2f} {n_iterations}")

## 📊 Visualize Random Search Exploration

In [ ]:
# Analyze random search results
random_results_df = pd.DataFrame(random_search.cv_results_)

# Plot score distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Random Search: Parameter Impact Analysis', fontsize=16, fontweight='bold')

# Plot 1: n_estimators vs score
axes[0, 0].scatter(random_results_df['param_n_estimators'], 
                   random_results_df['mean_test_score'],
                   alpha=0.6, c=random_results_df['mean_test_score'], 
                   cmap='viridis', s=100)
axes[0, 0].set_xlabel('n_estimators', fontsize=11)
axes[0, 0].set_ylabel('Mean CV Score', fontsize=11)
axes[0, 0].set_title('n_estimators Impact', fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# Plot 2: max_depth vs score
axes[0, 1].scatter(random_results_df['param_max_depth'], 
                   random_results_df['mean_test_score'],
                   alpha=0.6, c=random_results_df['mean_test_score'], 
                   cmap='viridis', s=100)
axes[0, 1].set_xlabel('max_depth', fontsize=11)
axes[0, 1].set_ylabel('Mean CV Score', fontsize=11)
axes[0, 1].set_title('max_depth Impact', fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Plot 3: min_samples_split vs score
axes[1, 0].scatter(random_results_df['param_min_samples_split'], 
                   random_results_df['mean_test_score'],
                   alpha=0.6, c=random_results_df['mean_test_score'], 
                   cmap='viridis', s=100)
axes[1, 0].set_xlabel('min_samples_split', fontsize=11)
axes[1, 0].set_ylabel('Mean CV Score', fontsize=11)
axes[1, 0].set_title('min_samples_split Impact', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Plot 4: Score distribution over iterations
sorted_scores = sorted(random_results_df['mean_test_score'], reverse=True)
axes[1, 1].plot(range(len(sorted_scores)), sorted_scores, 'b-', linewidth=2)
axes[1, 1].axhline(y=random_search.best_score_, color='r', linestyle='--', 
                   linewidth=2, label=f'Best: {random_search.best_score_:.4f}')
axes[1, 1].fill_between(range(len(sorted_scores)), sorted_scores, 
                        alpha=0.3)
axes[1, 1].set_xlabel('Iteration (sorted)', fontsize=11)
axes[1, 1].set_ylabel('Mean CV Score', fontsize=11)
axes[1, 1].set_title('Score Distribution', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Insights:")
print(f"   - Top 10% of configurations: {sorted_scores[0]:.4f} - {sorted_scores[4]:.4f}")
print(f"   - Worst configuration: {sorted_scores[-1]:.4f}")
print(f"   - Improvement range: {(sorted_scores[0] - sorted_scores[-1]):.4f}")
print(f"   → Hyperparameter tuning made a {(sorted_scores[0] - sorted_scores[-1])*100:.2f}% difference!")

## 🤖 Real AI Example: Optimizing a Multimodal Classifier

**Scenario:** You're building a classifier for a multimodal AI system (combining text and image features).

**Goal:** Find the best Gradient Boosting model for this task.

**Real-world application:**
- Image + caption classification (Instagram, Pinterest)
- Product categorization (Amazon, eBay)
- Content moderation with context

In [ ]:
# Simulate multimodal feature data
print("🖼️ + 📝 MULTIMODAL AI CLASSIFIER OPTIMIZATION")
print("=" * 60)

# Create a challenging dataset
X_multi, y_multi = make_classification(
    n_samples=3000,
    n_features=100,  # 50 image features + 50 text features
    n_informative=80,
    n_redundant=20,
    n_classes=3,  # 3 categories
    n_clusters_per_class=2,
    weights=[0.5, 0.3, 0.2],  # Imbalanced
    random_state=42
)

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

print(f"Dataset: {len(X_multi)} samples, {X_multi.shape[1]} features (multimodal)")
print(f"Classes: {np.bincount(y_multi)}")
print(f"Training set: {len(X_train_m)} samples")
print(f"Test set: {len(X_test_m)} samples\n")

# Gradient Boosting hyperparameters to tune
param_dist = {
    'n_estimators': randint(50, 300),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'subsample': uniform(0.6, 0.4),  # 0.6 to 1.0
    'max_features': uniform(0.5, 0.5)  # 0.5 to 1.0
}

print("🎛️  Hyperparameters to optimize:")
for param in param_dist.keys():
    print(f"   - {param}")

# Random search with more iterations
print("\n⏳ Running comprehensive Random Search (100 iterations)...\n")

random_search_gb = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_dist,
    n_iter=100,
    cv=5,
    scoring='f1_macro',  # Macro F1 for multiclass
    n_jobs=-1,
    verbose=1,
    random_state=42
)

start = time.time()
random_search_gb.fit(X_train_m, y_train_m)
tuning_time = time.time() - start

print(f"\n✅ Tuning completed in {tuning_time:.2f} seconds ({tuning_time/60:.2f} minutes)\n")

# Best model
best_model = random_search_gb.best_estimator_

# Evaluate on test set
y_pred = best_model.predict(X_test_m)

print("=" * 60)
print("🏆 BEST HYPERPARAMETERS:")
print("=" * 60)
for param, value in random_search_gb.best_params_.items():
    if isinstance(value, float):
        print(f"  {param:<20}: {value:.4f}")
    else:
        print(f"  {param:<20}: {value}")

print("\n" + "=" * 60)
print("📊 PERFORMANCE COMPARISON:")
print("=" * 60)

# Compare default vs tuned
default_model = GradientBoostingClassifier(random_state=42)
default_model.fit(X_train_m, y_train_m)
default_score = default_model.score(X_test_m, y_test_m)
tuned_score = best_model.score(X_test_m, y_test_m)

print(f"\nDefault parameters:  {default_score:.4f}")
print(f"Tuned parameters:    {tuned_score:.4f}")
print(f"Improvement:         {(tuned_score - default_score):.4f} ({(tuned_score - default_score)*100:.2f}%)")

# Detailed classification report
print("\n" + "=" * 60)
print("📋 DETAILED CLASSIFICATION REPORT (Tuned Model):")
print("=" * 60)
print(classification_report(y_test_m, y_pred, target_names=['Class 0', 'Class 1', 'Class 2']))

# Confusion matrix
cm = confusion_matrix(y_test_m, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=['Class 0', 'Class 1', 'Class 2'],
           yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix: Tuned Multimodal Classifier', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

print("\n💡 Production Readiness:")
if tuned_score > 0.85:
    print("   ✅ Model performance is excellent!")
    print("   ✅ Ready for A/B testing in production")
elif tuned_score > 0.75:
    print("   ⚠️  Good performance, but could be better")
    print("   → Consider feature engineering or more data")
else:
    print("   ❌ Needs improvement before production")
    print("   → Try different models or collect more data")

print(f"\n🎯 ROI of Hyperparameter Tuning:")
print(f"   Time invested: {tuning_time/60:.1f} minutes")
print(f"   Performance gain: {(tuned_score - default_score)*100:.2f}%")
print(f"   → Worth it! ✅")

## 📋 Best Practices for Hyperparameter Tuning

**1. Start with Random Search**
- Faster exploration
- Good for initial search
- Then refine with Grid Search if needed

**2. Use Cross-Validation**
- ALWAYS use CV, never just train-test split
- 5-fold is usually sufficient
- Stratified for classification

**3. Choose the Right Metric**
- Match your business goal
- Imbalanced data: F1, not accuracy
- Multi-class: macro-F1 or weighted-F1

**4. Search Space Design**
- Start wide, then narrow down
- Use log-scale for learning rates: [0.001, 0.01, 0.1, 1.0]
- Know which parameters matter most

**5. Computational Budget**
- Grid Search: Good for ≤3 parameters
- Random Search: Good for any number
- Consider time vs improvement trade-off

**6. Avoid Overfitting**
- Hold out a final test set
- Don't tune on test set!
- Use nested CV for unbiased estimates

**7. Parameter Importance**

**Random Forest:**
- Most important: `n_estimators`, `max_depth`, `min_samples_split`
- Less important: `min_samples_leaf`, `max_features`

**Gradient Boosting:**
- Most important: `learning_rate`, `n_estimators`, `max_depth`
- Less important: `subsample`, `min_samples_split`

**Neural Networks:**
- Most important: `learning_rate`, `batch_size`, `architecture`
- Less important: `optimizer choice`, `weight initialization`

## 🎯 YOUR TURN: Tune a Support Vector Machine

**Challenge:** Optimize an SVM classifier for the breast cancer dataset.

**Tasks:**
1. Define parameter distributions for SVM
2. Use RandomizedSearchCV with 50 iterations
3. Compare default vs tuned performance
4. Visualize the results

In [ ]:
# Load data
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("🎯 YOUR CHALLENGE: Tune an SVM Classifier")
print("=" * 60)

# YOUR CODE HERE!

# 1. Define parameter distributions for SVM
# Hint: Important SVM parameters are C, gamma, kernel
param_dist = {
    'C': # YOUR CODE (try uniform(0.1, 10))
    'gamma': # YOUR CODE (try uniform(0.001, 0.1))
    'kernel': # YOUR CODE (try ['rbf', 'linear'])
}

# 2. Create RandomizedSearchCV
random_search_svm = RandomizedSearchCV(
    # YOUR CODE
    n_iter=50,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

# 3. Fit and evaluate
# YOUR CODE

# 4. Compare with default
# YOUR CODE

# 5. Print results
# YOUR CODE

### ✅ Solution (Run after trying!)

In [ ]:
# SOLUTION
from sklearn.preprocessing import StandardScaler

print("🎯 SOLUTION: SVM Hyperparameter Tuning")
print("=" * 60)

# Important: Scale features for SVM!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 1. Define parameter distributions
param_dist_svm = {
    'C': uniform(0.1, 10),           # Regularization
    'gamma': uniform(0.001, 0.1),    # Kernel coefficient
    'kernel': ['rbf', 'linear']      # Kernel type
}

print("\n🔧 Parameter Space:")
for param, dist in param_dist_svm.items():
    print(f"   {param}: {dist}")

# 2. Random Search
print("\n⏳ Running Random Search (50 iterations)...\n")

random_search_svm = RandomizedSearchCV(
    SVC(random_state=42),
    param_dist_svm,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

start = time.time()
random_search_svm.fit(X_train_scaled, y_train)
svm_time = time.time() - start

print(f"\n✅ Tuning completed in {svm_time:.2f} seconds\n")

# 3. Best parameters
print("=" * 60)
print("🏆 BEST HYPERPARAMETERS:")
print("=" * 60)
for param, value in random_search_svm.best_params_.items():
    if isinstance(value, float):
        print(f"  {param:<10}: {value:.6f}")
    else:
        print(f"  {param:<10}: {value}")

# 4. Compare default vs tuned
default_svm = SVC(random_state=42)
default_svm.fit(X_train_scaled, y_train)
default_score = default_svm.score(X_test_scaled, y_test)

tuned_score = random_search_svm.score(X_test_scaled, y_test)

print("\n" + "=" * 60)
print("📊 PERFORMANCE COMPARISON:")
print("=" * 60)
print(f"\nDefault SVM:  {default_score:.4f}")
print(f"Tuned SVM:    {tuned_score:.4f}")
print(f"Improvement:  {(tuned_score - default_score):.4f} ({(tuned_score - default_score)*100:.2f}%)")

# 5. Visualize results
results_svm = pd.DataFrame(random_search_svm.cv_results_)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: C vs Score (for RBF kernel)
rbf_results = results_svm[results_svm['param_kernel'] == 'rbf']
ax1.scatter(rbf_results['param_C'], rbf_results['mean_test_score'], 
           c=rbf_results['param_gamma'], cmap='viridis', s=100, alpha=0.6)
ax1.set_xlabel('C (Regularization)', fontsize=12)
ax1.set_ylabel('Mean CV Accuracy', fontsize=12)
ax1.set_title('SVM: C vs Accuracy (RBF kernel)', fontsize=13, fontweight='bold')
ax1.grid(alpha=0.3)
cbar1 = plt.colorbar(ax1.collections[0], ax=ax1)
cbar1.set_label('gamma', fontsize=10)

# Plot 2: Kernel comparison
kernel_scores = results_svm.groupby('param_kernel')['mean_test_score'].agg(['mean', 'std'])
kernel_scores.plot(kind='bar', y='mean', yerr='std', ax=ax2, 
                  color=['skyblue', 'orange'], alpha=0.7, capsize=5)
ax2.set_xlabel('Kernel Type', fontsize=12)
ax2.set_ylabel('Mean CV Accuracy', fontsize=12)
ax2.set_title('Kernel Comparison', fontsize=13, fontweight='bold')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)
ax2.grid(axis='y', alpha=0.3)
ax2.legend(['Mean Score'], loc='lower right')

plt.tight_layout()
plt.show()

print("\n💡 Key Findings:")
print(f"   - Best kernel: {random_search_svm.best_params_['kernel']}")
print(f"   - Optimal C: {random_search_svm.best_params_['C']:.4f}")
if 'gamma' in random_search_svm.best_params_:
    print(f"   - Optimal gamma: {random_search_svm.best_params_['gamma']:.6f}")
print(f"   - Performance gain: {(tuned_score - default_score)*100:.2f}%")
print("\n🎯 Hyperparameter tuning made a significant difference! ✅")

## 🚀 Advanced: Comparing Multiple Models

**Real-world workflow:** Try multiple models, tune each, then compare!

In [ ]:
print("🏁 FINAL SHOWDOWN: Multiple Tuned Models")
print("=" * 60)

# Define models and their parameter spaces
models_to_tune = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': randint(50, 200),
            'max_depth': randint(5, 20),
            'min_samples_split': randint(2, 10)
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {
            'n_estimators': randint(50, 200),
            'learning_rate': uniform(0.01, 0.2),
            'max_depth': randint(3, 10)
        }
    },
    'SVM': {
        'model': SVC(random_state=42),
        'params': {
            'C': uniform(0.1, 10),
            'gamma': uniform(0.001, 0.1),
            'kernel': ['rbf', 'linear']
        }
    }
}

# Tune each model
results_comparison = {}

for name, config in models_to_tune.items():
    print(f"\n⚙️  Tuning {name}...")
    
    search = RandomizedSearchCV(
        config['model'],
        config['params'],
        n_iter=30,
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42
    )
    
    # Use scaled data for SVM
    if name == 'SVM':
        search.fit(X_train_scaled, y_train)
        test_score = search.score(X_test_scaled, y_test)
    else:
        search.fit(X_train, y_train)
        test_score = search.score(X_test, y_test)
    
    results_comparison[name] = {
        'best_params': search.best_params_,
        'cv_score': search.best_score_,
        'test_score': test_score,
        'cv_std': search.cv_results_['std_test_score'][search.best_index_]
    }
    
    print(f"   CV Score: {search.best_score_:.4f}")
    print(f"   Test Score: {test_score:.4f}")

# Display final comparison
print("\n" + "=" * 60)
print("🏆 FINAL MODEL COMPARISON")
print("=" * 60)
print(f"\n{'Model':<20} {'CV Score':<12} {'Test Score':<12} {'Std'}")
print("-" * 60)

for name, res in results_comparison.items():
    print(f"{name:<20} {res['cv_score']:.4f}       {res['test_score']:.4f}       {res['cv_std']:.4f}")

# Find best model
best_model_name = max(results_comparison.keys(), key=lambda k: results_comparison[k]['test_score'])
best_result = results_comparison[best_model_name]

print("\n" + "=" * 60)
print(f"🥇 WINNER: {best_model_name}")
print("=" * 60)
print(f"Test Accuracy: {best_result['test_score']:.4f}")
print(f"Best Parameters:")
for param, value in best_result['best_params'].items():
    if isinstance(value, float):
        print(f"  {param}: {value:.6f}")
    else:
        print(f"  {param}: {value}")

# Visualize comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

models = list(results_comparison.keys())
cv_scores = [results_comparison[m]['cv_score'] for m in models]
test_scores = [results_comparison[m]['test_score'] for m in models]
stds = [results_comparison[m]['cv_std'] for m in models]

# Plot 1: CV vs Test scores
x = np.arange(len(models))
width = 0.35

ax1.bar(x - width/2, cv_scores, width, label='CV Score', alpha=0.7, color='skyblue')
ax1.bar(x + width/2, test_scores, width, label='Test Score', alpha=0.7, color='orange')
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Model Comparison: CV vs Test', fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(models)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Test scores with error bars
ax2.bar(models, test_scores, yerr=stds, capsize=10, alpha=0.7, 
       color=['#3498db', '#2ecc71', '#e74c3c'])
ax2.set_ylabel('Test Accuracy', fontsize=12)
ax2.set_title('Test Performance with Std Dev', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

for i, (score, std) in enumerate(zip(test_scores, stds)):
    ax2.text(i, score + std + 0.005, f'{score:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🎯 Production Recommendation:")
print(f"   Deploy: {best_model_name}")
print(f"   Expected accuracy: {best_result['test_score']:.2%}")
print(f"   Confidence: High ✅ (based on CV consistency)")

## 🎉 Congratulations!

**You just mastered:**
- ✅ What hyperparameters are and why they matter
- ✅ Grid Search: exhaustive but slow
- ✅ Random Search: smart and efficient
- ✅ How to use GridSearchCV and RandomizedSearchCV
- ✅ Best practices for hyperparameter tuning
- ✅ Real AI application: multimodal classifier optimization
- ✅ Comparing multiple tuned models

**🎯 Key Takeaways:**
1. **Default parameters are rarely optimal** - always tune!
2. **Random Search > Grid Search** for most cases
3. **Use cross-validation** during tuning
4. **Start wide, then narrow** your search space
5. **Compare multiple models** after tuning each
6. **Know which parameters matter** for each model type

**📊 Week 8 Complete! You Now Know:**

**Day 1:** Evaluation Metrics
- Accuracy, Precision, Recall, F1-Score
- Confusion Matrix, ROC, AUC
- When to use which metric

**Day 2:** Cross-Validation
- K-Fold, Stratified K-Fold, LOOCV
- Reliable performance estimates
- Avoiding lucky/unlucky splits

**Day 3:** Hyperparameter Tuning
- Grid Search vs Random Search
- Systematic optimization
- Production-ready model selection

**🚀 Final Practice Project:**

Build a complete ML pipeline:
1. Load a dataset of your choice
2. Try 3 different models
3. Tune each with RandomizedSearchCV
4. Evaluate with proper metrics (precision, recall, F1)
5. Use Stratified 5-Fold CV
6. Compare and choose the best
7. Create a final evaluation report

---

**📚 Next Week:** Week 9 - Feature Engineering & Selection

**💬 Questions?** Experiment with different search spaces and see how results change!

---

*"The difference between a good model and a great model is often just... proper hyperparameter tuning!"* 🎛️

**🌟 You're now ready to build production-quality ML models! Congratulations! 🌟**